## Part C — Comprehensive Analysis



Purpose: Hyperparameter analysis and model comparison (Logistic, Softmax, Neural Network) on MNIST. This notebook reuses code from Part A and Part B provided by the student.


## Instructions

This notebook expects the DataLoaders train_loader, val_loader, test_loader (flattened 28x28 -> 784) to be defined in the environment. It also uses the provided model implementations (Logistic, Softmax, CustomFeedforwardNN) and training helpers.

Run cells in order.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import os , sys
sys.path.append(os.path.abspath("../"))
from src.logisitc_manual import LogisticRegression
from src.softmax_manual import SoftmaxRegression

# For reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

### Logistic Regression (Part A2)


In [ ]:


def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def binary_cross_entropy(y_pred, y_true):
    eps = 1e-8
    return -torch.mean(y_true * torch.log(y_pred + eps) + (1 - y_true) * torch.log(1 - y_pred + eps))

def forward_pass(X_batch, W, b):
    scores = X_batch @ W + b
    y_pred = sigmoid(scores)
    return y_pred

def compute_accuracy(y_pred, y_true):
    threshold = (y_pred >= 0.5)
    predictions = threshold.float()
    return (predictions.squeeze() == y_true).float().mean().item()

def compute_gradients(X_batch, y_batch, y_pred):
    error = y_pred - y_batch.unsqueeze(1)
    dW = (X_batch.T @ error) / X_batch.shape[0]
    db = error.mean()
    return dW, db

def update_weights(W, b, dW, db, alpha):
    W -= alpha * dW
    b -= alpha * db
    return W, b

def train_logistic(train_loader, val_loader, learning_rate=0.01, max_epochs=100, patience=10, min_delta=0.0005, early_stopping=True):
    n_features = 28*28
    W = torch.zeros(n_features, 1)
    b = torch.zeros(1)
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    best_val_loss = float('inf'); epochs_no_improve = 0
    best_W, best_b = None, None
    for epoch in range(max_epochs):
        train_loss_epoch = 0.0; train_acc_epoch = 0.0; n_train_batches = 0
        for X_batch, y_batch in train_loader:
            y_pred = forward_pass(X_batch, W, b)
            loss = binary_cross_entropy(y_pred, y_batch.unsqueeze(1))
            dW, db = compute_gradients(X_batch, y_batch, y_pred)
            W, b = update_weights(W, b, dW, db, learning_rate)
            train_loss_epoch += loss.item(); train_acc_epoch += compute_accuracy(y_pred, y_batch)
            n_train_batches += 1
        val_loss_epoch = 0.0; val_acc_epoch = 0.0; n_val = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                y_pred = forward_pass(X_batch, W, b)
                loss = binary_cross_entropy(y_pred, y_batch.unsqueeze(1))
                val_loss_epoch += loss.item(); val_acc_epoch += compute_accuracy(y_pred, y_batch)
                n_val += 1
        avg_train_loss = train_loss_epoch / n_train_batches
        avg_val_loss = val_loss_epoch / n_val
        train_losses.append(avg_train_loss); val_losses.append(avg_val_loss)
        train_accs.append(train_acc_epoch / n_train_batches); val_accs.append(val_acc_epoch / n_val)
        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{max_epochs} - Train Loss: {train_losses[-1]:.4f} Train Acc: {train_accs[-1]:.4f} - Val Loss: {val_losses[-1]:.4f} Val Acc: {val_accs[-1]:.4f}")
        # early stopping
        if early_stopping:
            if avg_val_loss < best_val_loss - min_delta:
                best_val_loss = avg_val_loss; epochs_no_improve = 0
                best_W = W.clone(); best_b = b.clone()
            else:
                epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    if early_stopping and best_W is not None:
        W = best_W; b = best_b
    return W, b, train_losses, val_losses, train_accs, val_accs


## Softmax Regression (Part A3)

In [ ]:
class SoftmaxRegression:
    def __init__(self, input_dim, num_classes, learning_rate=0.01, reg_lambda=0.0):
        self.input_dim = input_dim; self.num_classes = num_classes; self.lr = learning_rate; self.reg_lambda = reg_lambda
        self.W = torch.randn(self.input_dim, self.num_classes) * 0.01
        self.b = torch.zeros(self.num_classes)
        self.history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    def softmax(self, z):
        exp_z = torch.exp(z - torch.max(z, dim=1, keepdim=True)[0])
        return exp_z / torch.sum(exp_z, dim=1, keepdim=True)
    def cross_entropy_loss(self, y_pred, y_true):
        eps = 1e-8
        y_true_one_hot = torch.nn.functional.one_hot(y_true.long(), num_classes=self.num_classes).float()
        loss = -torch.mean(torch.sum(y_true_one_hot * torch.log(y_pred + eps), dim=1))
        return loss
    def predict_probabilities(self, X_batch):
        scores = X_batch @ self.W + self.b
        y_pred = self.softmax(scores)
        return y_pred
    def predict(self, X_batch):
        probs = self.predict_probabilities(X_batch)
        return torch.argmax(probs, dim=1)
    def compute_accuracy(self, y_pred, y_true):
        return torch.mean((y_pred == y_true).float())
    def fit(self, train_loader, val_loader=None, epochs=100):
        for epoch in range(epochs):
            total_loss, correct, total = 0.0, 0, 0
            for X_batch, y_batch in train_loader:
                probs = self.predict_probabilities(X_batch)
                loss = self.cross_entropy_loss(probs, y_batch)
                y_true_one_hot = torch.nn.functional.one_hot(y_batch.long(), num_classes=self.num_classes).float()
                grad_scores = (probs - y_true_one_hot) / X_batch.shape[0]
                grad_W = X_batch.T @ grad_scores + self.reg_lambda * self.W
                grad_b = torch.sum(grad_scores, dim=0)
                self.W -= self.lr * grad_W
                self.b -= self.lr * grad_b
                total_loss += loss.item()
                preds = torch.argmax(probs, dim=1)
                correct += torch.sum(preds == y_batch).item()
                total += y_batch.size(0)
            train_loss = total_loss / len(train_loader); train_acc = correct / total
            self.history['train_loss'].append(train_loss); self.history['train_acc'].append(train_acc)
            if val_loader is not None:
                val_loss, val_acc = self.evaluate(val_loader)
                self.history['val_loss'].append(val_loss); self.history['val_acc'].append(val_acc)
                if (epoch + 1) % 10 == 0:
                    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {self.history['train_loss'][-1]:.4f}, Train Acc: {self.history['train_acc'][-1]:.4f} - Val Loss: {self.history['val_loss'][-1]:.4f}, Val Acc: {self.history['val_acc'][-1]:.4f}")
            else:
                print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    def evaluate(self, data_loader):
        total_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for X_batch, y_batch in data_loader:
                probs = self.predict_probabilities(X_batch)
                loss = self.cross_entropy_loss(probs, y_batch)
                total_loss += loss.item()
                preds = torch.argmax(probs, dim=1)
                correct += torch.sum(preds == y_batch).item()
                total += y_batch.size(0)
        return total_loss / len(data_loader), correct / total


## Neural network (Part B)

In [ ]:
class CustomFeedforwardNN(nn.Module):
    def __init__(self, input_size=784, hidden1_size=128, hidden2_size=64, output_size=10):
        super(CustomFeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, output_size)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        nn.init.zeros_(self.fc3.bias)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training loop for neural network (Part B)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model_once(model, train_loader, val_loader, epochs=10, learning_rate=0.01, device=device):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_loss = running_loss / total
        train_acc = correct / total
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        val_loss /= val_total
        val_acc = val_correct / val_total
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        print(f"Epoch {epoch+1}/{epochs} Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f}")
    return train_losses, val_losses, train_accuracies, val_accuracies

# Multiple-run helper to compute mean +/- std
def train_multiple_times(model_class, train_loader, val_loader, epochs=10, learning_rate=0.01, runs=5):
    all_train_losses, all_val_losses = [], []
    all_train_acc, all_val_acc = [], []
    for r in range(runs):
        print(f"\nRun {r+1}/{runs}")
        model = model_class()
        tl, vl, ta, va = train_model_once(model, train_loader, val_loader, epochs, learning_rate, device)
        all_train_losses.append(tl); all_val_losses.append(vl)
        all_train_acc.append(ta); all_val_acc.append(va)
    all_train_losses = np.array(all_train_losses)
    all_val_losses = np.array(all_val_losses)
    all_train_acc = np.array(all_train_acc)
    all_val_acc = np.array(all_val_acc)
    mean_train_loss = np.mean(all_train_losses, axis=0)
    std_train_loss = np.std(all_train_losses, axis=0)
    mean_val_loss = np.mean(all_val_losses, axis=0)
    std_val_loss = np.std(all_val_losses, axis=0)
    mean_train_acc = np.mean(all_train_acc, axis=0)
    std_train_acc = np.std(all_train_acc, axis=0)
    mean_val_acc = np.mean(all_val_acc, axis=0)
    std_val_acc = np.std(all_val_acc, axis=0)
    epochs_range = range(1, epochs+1)
    plt.figure(figsize=(10,5))
    plt.errorbar(epochs_range, mean_train_loss, yerr=std_train_loss, label='Training Loss')
    plt.errorbar(epochs_range, mean_val_loss, yerr=std_val_loss, label='Validation Loss')
    plt.title("Loss over Epochs with Error Bars")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(); plt.show()
    plt.figure(figsize=(10,5))
    plt.errorbar(epochs_range, mean_train_acc, yerr=std_train_acc, label='Training Acc')
    plt.errorbar(epochs_range, mean_val_acc, yerr=std_val_acc, label='Validation Acc')
    plt.title("Accuracy over Epochs with Error Bars")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(); plt.show()
    val_loss_diff = np.abs(np.diff(mean_val_loss))
    convergence_epoch = np.argmin(val_loss_diff) + 1 if len(val_loss_diff)>0 else 1
    print(f"Convergence epoch (approx): {convergence_epoch}")
    return {
        'mean_train_loss': mean_train_loss, 'std_train_loss': std_train_loss,
        'mean_val_loss': mean_val_loss, 'std_val_loss': std_val_loss,
        'mean_train_acc': mean_train_acc, 'std_train_acc': std_train_acc,
        'mean_val_acc': mean_val_acc, 'std_val_acc': std_val_acc
    }


## C1 — Hyperparameter Analysis

### C1.1 Learning Rate Analysis


In [ ]:
learning_rates = [0.001, 0.01, 0.1, 1.0]
num_epochs = 20
batch_size = 64
results_lr = []
for lr in learning_rates:
    print(f"\n=== LR = {lr} ===")
    # ensure we use DataLoaders with the chosen batch size
    # If your original train_loader has different batch size, recreate it from tensors if available
    # Here we assume train_loader and val_loader accept any batch sizes; if not, recreate from TensorDataset.
    model = CustomFeedforwardNN()
    tl, vl, ta, va = train_model_once(model, train_loader, val_loader, epochs=num_epochs, learning_rate=lr)
    results_lr.append({'lr': lr, 'train_losses': tl, 'val_losses': vl, 'train_acc': ta, 'val_acc': va})

# Plot validation loss comparison
plt.figure(figsize=(8,5))
for res in results_lr:
    plt.plot(res['val_losses'], label=f"lr={res['lr']}")
plt.xlabel('Epoch'); plt.ylabel('Val Loss'); plt.title('Validation Loss for different learning rates'); plt.legend(); plt.grid(True); plt.show()

# Summarize final val accuracy
pd.DataFrame([{'lr': r['lr'], 'final_val_acc': r['val_acc'][-1]} for r in results_lr])


### C1.2 Batch Size Analysis


In [ ]:
batch_sizes = [16, 32, 64, 128]
num_epochs = 20
results_bs = []
# If tensors for X_train_flat etc exist, recreate DataLoaders with different batch sizes
try:
    X_train_flat
    recreate_loaders = True
except NameError:
    recreate_loaders = False

for bs in batch_sizes:
    print(f"\n=== Batch size = {bs} ===")
    if recreate_loaders:
        train_loader_bs = DataLoader(TensorDataset(X_train_flat, y_train), batch_size=bs, shuffle=True)
        val_loader_bs = DataLoader(TensorDataset(X_val_flat, y_val), batch_size=bs, shuffle=False)
    else:
        train_loader_bs = train_loader; val_loader_bs = val_loader
    model = CustomFeedforwardNN()
    start = time.time()
    tl, vl, ta, va = train_model_once(model, train_loader_bs, val_loader_bs, epochs=num_epochs, learning_rate=0.01)
    duration = time.time() - start
    results_bs.append({'bs': bs, 'train_losses': tl, 'val_losses': vl, 'train_acc': ta, 'val_acc': va, 'time_s': duration})

plt.figure(figsize=(8,5))
for res in results_bs:
    plt.plot(res['val_losses'], label=f"bs={res['bs']}")
plt.xlabel('Epoch'); plt.ylabel('Val Loss'); plt.title('Validation Loss for different batch sizes'); plt.legend(); plt.grid(True); plt.show()

pd.DataFrame([{'batch_size': r['bs'], 'final_val_acc': r['val_acc'][-1], 'time_s': r['time_s']} for r in results_bs])

### C1.3 Architecture Analysis


In [ ]:
def make_ffnn(hidden_sizes):
    # hidden_sizes: list of ints
    if len(hidden_sizes) == 1:
        return CustomFeedforwardNN(input_size=784, hidden1_size=hidden_sizes[0], hidden2_size=64, output_size=10)
    elif len(hidden_sizes) == 2:
        return CustomFeedforwardNN(input_size=784, hidden1_size=hidden_sizes[0], hidden2_size=hidden_sizes[1], output_size=10)
    else:
        # for >2 layers create dynamic module
        class FFN_dynamic(nn.Module):
            def __init__(self, input_dim=784, hidden_sizes=[128,64], output_dim=10):
                super().__init__()
                layers = []
                prev = input_dim
                for h in hidden_sizes:
                    layers.append(nn.Linear(prev, h))
                    layers.append(nn.ReLU())
                    prev = h
                layers.append(nn.Linear(prev, output_dim))
                self.net = nn.Sequential(*layers)
                for m in self.net:
                    if isinstance(m, nn.Linear):
                        nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)
            def forward(self, x):
                return self.net(x)
        return FFN_dynamic(input_dim=784, hidden_sizes=hidden_sizes, output_dim=10)

layers_options = [ [64,64], [128,64], [256,128,64], [512,256,128,64] ]
results_arch = []
for hidden in layers_options:
    print(f"\n=== Arch: {hidden} ===")
    model = make_ffnn(hidden)
    tl, vl, ta, va = train_model_once(model, train_loader, val_loader, epochs=20, learning_rate=0.01)
    results_arch.append({'hidden': hidden, 'train_acc': ta[-1], 'val_acc': va[-1], 'train_losses': tl, 'val_losses': vl})

pd.DataFrame([{'architecture': str(r['hidden']), 'train_acc': r['train_acc'], 'val_acc': r['val_acc']} for r in results_arch])


## C2 — Model Comparison


In [ ]:
# Softmax Regression (multiclass)
soft_model = SoftmaxRegression(input_dim=784, num_classes=10, learning_rate=0.1)
soft_model.fit(train_loader, val_loader, epochs=50)
val_loss_soft, val_acc_soft = soft_model.evaluate(val_loader)
print("Softmax val acc:", val_acc_soft)

# Neural network: choose best from architecture experiments (example pick index 1)
best_hidden = results_arch[1]['hidden'] if len(results_arch)>1 else [128,64]
best_model = make_ffnn(best_hidden)
start = time.time()
tl, vl, ta, va = train_model_once(best_model, train_loader, val_loader, epochs=30, learning_rate=0.01)
nn_time = time.time() - start
# Evaluate on test set utility

def evaluate_nn_on_loader(model, loader):
    model.to(device); model.eval()
    correct = 0; total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            _, preds = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            correct += (preds.cpu() == y_batch).sum().item()
            total += y_batch.size(0)
    return correct/total, np.array(y_true), np.array(y_pred)

# Evaluate Softmax on test
soft_test_loss, soft_test_acc = soft_model.evaluate(test_loader)
print('Softmax test acc:', soft_test_acc)

# Evaluate NN on test
nn_test_acc, y_true_nn, y_pred_nn = evaluate_nn_on_loader(best_model, test_loader)
print('NN test acc:', nn_test_acc)

# Logistic: if binary problem, train logistic regression
# If you want to compare on full 10-class, logistic is not directly used (would require one-vs-rest); we keep logistic for binary experiments.

# Summary table
comparison = pd.DataFrame([
    {'Model': 'Softmax Regression', 'Test Accuracy': float(soft_test_acc), 'Training Time (s)': float('nan')},
    {'Model': f'Neural Network {best_hidden}', 'Test Accuracy': float(nn_test_acc), 'Training Time (s)': nn_time}
])
comparison

## C2.1 Confusion Matrix & Misclassified Examples (Best NN)


In [ ]:
cm = confusion_matrix(y_true_nn, y_pred_nn)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(8,8))
disp.plot(ax=ax)
plt.title('Confusion Matrix - Best NN')
plt.show()

# Show some misclassified examples
mis_idx = np.where(y_true_nn != y_pred_nn)[0]
print('Total misclassified:', len(mis_idx))

# Grab images from test dataset (original test dataset should be available as X_test / y_test or test_loader.dataset)
# Assuming test_loader.dataset is a TensorDataset of flattened images

dataset_for_vis = test_loader.dataset
fig, axes = plt.subplots(2,5, figsize=(12,5))
for i, ax in enumerate(axes.flat):
    if i >= len(mis_idx): break
    idx = mis_idx[i]
    # if dataset stores flattened images, reshape
    img_flat, true_label = dataset_for_vis[idx]
    if img_flat.ndim == 1:
        img = img_flat.reshape(28,28)
    else:
        img = img_flat.squeeze()
    ax.imshow(img, cmap='gray')
    ax.set_title(f"Pred: {y_pred_nn[idx]} / True: {y_true_nn[idx]}")
    ax.axis('off')
plt.show()

## C3 — Final Evaluation & Retraining Best Model on Train+Val


In [ ]:
# Combine train + val into one dataset and retrain best model
try:
    X_train_flat; X_val_flat; y_train; y_val
    X_train_comb = torch.cat([X_train_flat, X_val_flat], dim=0)
    y_train_comb = torch.cat([y_train, y_val], dim=0)
    combined_loader = DataLoader(TensorDataset(X_train_comb, y_train_comb), batch_size=64, shuffle=True)
    final_model = make_ffnn(best_hidden)
    # train
    train_model_once(final_model, combined_loader, test_loader, epochs=30, learning_rate=0.01)
    final_acc, y_true_final, y_pred_final = evaluate_nn_on_loader(final_model, test_loader)
    print('Final test acc after retraining on train+val:', final_acc)
except NameError:
    print('Train/Val tensors not found in workspace; skip final retrain step or recreate tensors.')

## Save results


In [ ]:
# Save best model weights (optional)
try:
    torch.save(final_model.state_dict(), 'best_model_final.pt')
    print('Saved best_model_final.pt')
except NameError:
    print('Final model not trained — no file saved')

## End of Notebook